### 🛠️ Day 8 Tasks:

1. Create catalog & schemas
2. Register Delta tables
3. Set up permissions
4. Create views for controlled access

## Create catalog & schemas

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ecommerce_catalog;

SHOW CATALOGS;

catalog
ecommerce_catalog
samples
system
workspace


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.bronze;
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.silver;
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.gold;

SHOW SCHEMAS IN ecommerce_catalog;


databaseName
bronze
default
gold
information_schema
silver


## Register Delta Tables

Read Raw Data

In [0]:
df_bronze = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/workspace/ecommerce/ecommerce_data/")
)


Register Bronze Table

In [0]:
df_bronze.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce_catalog.bronze.orders")

validate

In [0]:
%sql
SELECT COUNT(*) FROM ecommerce_catalog.bronze.orders;



COUNT(*)
109950743


Read Bronze table

In [0]:
from pyspark.sql.functions import col, to_timestamp

df_silver = (
    spark.read.table("ecommerce_catalog.bronze.orders")
    .filter(col("user_id").isNotNull())
    .filter(col("price") > 0)
    .withColumn("event_time", to_timestamp("event_time"))
)


Register Silver table

In [0]:
df_silver.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce_catalog.silver.orders_clean")


validate

In [0]:
%sql
SELECT COUNT(*) FROM ecommerce_catalog.silver.orders_clean;



COUNT(*)
109693982


Register Gold Delta Table

Create Aggregates

In [0]:
from pyspark.sql.functions import sum, count, avg, to_date

df_gold = (
    spark.read.table("ecommerce_catalog.silver.orders_clean")
    .withColumn("order_date", to_date("event_time"))
    .groupBy("order_date")
    .agg(
        sum("price").alias("total_revenue"),
        count("*").alias("total_orders"),
        avg("price").alias("avg_order_value")
    )
)


Register Gold Table

In [0]:
df_gold.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce_catalog.gold.daily_sales")


validate

In [0]:
%sql
SELECT * FROM ecommerce_catalog.gold.daily_sales LIMIT 10;


order_date,total_revenue,total_orders,avg_order_value
2019-11-17,1.8764804914989712E9,6371159,294.5273366272873
2019-11-16,1.998441656940825E9,6460123,309.35040353578796
2019-11-20,4.6682197337990606E8,1695812,275.27931951177726
2019-11-18,5.851247946899247E8,2017005,290.09585731811507
2019-11-19,4.8991056515983117E8,1725089,283.9914724166876
2019-11-24,4.472729619498782E8,1591284,281.0767669063965
2019-11-21,4.483642548998058E8,1673897,267.85653770799865
2019-11-23,4.4019962935998046E8,1561182,281.96560641871383
2019-11-22,4.3793466844978905E8,1565600,279.72321694544524
2019-11-25,4.400030527599251E8,1592360,276.3213423848408


## Set Up Permissions

In [0]:
%sql
GRANT ALL PRIVILEGES
ON SCHEMA ecommerce_catalog.bronze
TO `karthika2738@gmail.com`;

GRANT ALL PRIVILEGES
ON SCHEMA ecommerce_catalog.silver
TO `karthika2738@gmail.com`;

GRANT SELECT
ON SCHEMA ecommerce_catalog.gold
TO `karthika2738@gmail.com`;



## Create views for controlled access

In [0]:
%sql
CREATE OR REPLACE VIEW ecommerce_catalog.gold.daily_sales_view AS
SELECT
  order_date,
  total_revenue,
  total_orders,
  avg_order_value
FROM ecommerce_catalog.gold.daily_sales;


In [0]:
%sql
GRANT SELECT
ON VIEW ecommerce_catalog.gold.daily_sales_view
TO `account users`;